### 고객 구매상품 관련 분석
    
    최초 작성일 : 20/03/02
    작성자 : 김은주
    
    작성내용 : 고객 구매상품 분석
    
               DB 內 PRDT_REVIEW 테이블 활용
               - 상품코드 기준 상품 대표 이미지(ex. 썸네일 이미지) 크롤링
               -> 이미지에서 대표 색상 추출하여 DB 추가
               
               유저 닉네임 별 주요 구매 색상 추출
               -> 

    수정내용 
        20/03/02

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

In [8]:
#DB 연결 데이터 
config = {"host":"49.142.181.65", "user":"cai", "passwd":"1234", "db":"final"}
conn = pymysql.connect(**config)
cursor = conn.cursor()

In [ ]:
sql = "SELECT *,round((A.L+A.A+A.S+A.C)/4, 5) as 'con_avg' FROM (SELECT b.BRD_NAME_ENG, PRODUCT, REVIEW, LIKES, LOOKBOOK/(SELECT sum(LOOKBOOK) FROM BRAND_INFO) as 'L', ARTICLE/(SELECT sum(ARTICLE) FROM BRAND_INFO) as 'A', STYLE/(SELECT sum(STYLE) FROM BRAND_INFO) as 'S', CONTENTS/(SELECT sum(CONTENTS) FROM BRAND_INFO) as 'C' FROM BRAND_INFO a, BRAND_LIST b WHERE a.BRD_NAME_ENG = b.BRD_NAME_ENG AND a.BRD_NAME_ENG != 'Disney' AND a.BRD_NAME_ENG != 'ABERCROMBIE & FITCH') A WHERE (A.L+A.A+A.S+A.C) != 0;"
cursor.execute(sql)
brand_info = pd.DataFrame(cursor, columns=["brand", "prdt", "review", "likes", "lookbook", "article", "style", "contents", "score"])

columns = ["prdt", "review", "likes", "lookbook", "article", "style", "contents", "score"]
for col in columns:
    brand_info[col] = brand_info[col].astype(float)

brand_info.info()

In [16]:
############################
# 웹에서 받아와야함
user_nick = str(input())
############################

sql_front = "SELECT c.H, c.S, c.V, b.PRDT_CODE, c.PRDT_IMG_LINK FROM PRDT_REVIEW a, FINAL_TOP b, PRDT_IMAGE c WHERE USER_NICK='"
sql_end = "' AND a.PRDT_CODE = b.PRDT_CODE AND a.PRDT_CODE = c.PRDT_CODE;"
sql = sql_front+user_nick+sql_end

cursor.execute(sql)
"""
↑↑↑↑유저 조회가 안돼서 발생하는 오류 잡을거 짜놓기
"""

# purchase_prdt = pd.DataFrame(cursor, columns=["H", "S", "V", "prdt", "img_link"])
# purchase_prdt

purchase_prdt = [] # 고객 구매목록 담기
for i in cursor.fetchall() :
    purchase_prdt.append(list(i))

purchase_prdt

JAY


[[None,
  None,
  None,
  '938783',
  'https://image.msscdn.net/images/goods_img/20190123/938783/938783_1_500.jpg'],
 [None,
  None,
  None,
  '428572',
  'https://image.msscdn.net/images/goods_img/20161013/428572/428572_2_500.jpg'],
 [None,
  None,
  None,
  '240543',
  'https://image.msscdn.net/images/goods_img/20150825/240543/240543_2_500.jpg'],
 [None,
  None,
  None,
  '846573',
  'https://image.msscdn.net/images/goods_img/20180904/846573/846573_2_500.jpg'],
 [None,
  None,
  None,
  '485070',
  'https://image.msscdn.net/images/goods_img/20170217/485070/485070_1_500.jpg']]